# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR TOKEN"

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    temperature=0
)

response = llm.invoke("Explain likedin in few words")
print(response.content)

Professional networking platform.


In [5]:
import pandas as pd
df = pd.read_csv('claims_report.csv')

In [6]:
df.head()

,Claims_NPI,Claims_Address1,Claims_Address2,Claims_City,Claims_State,Claims_Zip,Claims_DateOfService,STANDARDIZED ADDRESS1,STANDARDIZED ADDRESS2,STANDARDIZED CITY,STANDARDIZED STATE,STANDARDIZED ZIP,CLIENT
0,1356444897,FILE 57326,NaN,LOS ANGELES,CA,900747326,20240614,FILE 57326,NaN,LOS ANGELES,CA,900747326,Molina
1,1699776526,PO BOX 511412,NaN,Los Angeles,CA,900517967,20240614,PO BOX 511412,NaN,Los Angeles,CA,900517967,Molina
2,1508352675,550 N BRAND BLVD,STE 1000,GLENDALE,CA,91203,20240617,550 N BRAND BLVD,STE 1000,GLENDALE,CA,91203,Molina
3,1669471603,9808 VENICE BLVD,STE 503,CULVER CITY,CA,90232-6804,20240618,9808 VENICE BLVD,STE 503,CULVER CITY,CA,90232-6804,Molina
4,1699776526,PO BOX 511412,NaN,Los Angeles,CA,900517967,20240619,PO BOX 511412,NaN,Los Angeles,CA,900517967,Molina


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain

In [21]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [22]:
chain = prompt | llm

In [24]:
product = "Queen size sheet set"
result = chain.invoke({'product': {product}})

print(result.content)

The best name depends on the desired brand image. Here are a few options, categorized by the image they project:

**Luxury/High-End:**

* Royal Rest
* Empress Linens
* Sovereign Sheets
* The Silk Thread
* Celestial Sleep

**Cozy/Comfortable:**

* Cozy Nights
* Slumber Haven
* Sweet Dreams Sheets
* The Comfy Co.
* Restful Retreat

**Simple/Direct:**

* Queen Sheets
* Premium Bedding
* The Sheet Shop
* Bedtime Bliss
* Simply Sheets

**Modern/Minimalist:**

* Linen & Co.
* Thread & Loom
* The Sleep Edit
* Rest & Renew
* Slumber

**Unique/Creative:**

* Dream Weaver Linens
* Midnight Bloom
* The Cotton Cloud
* Whispering Sheets
* Moonlit Slumber


When choosing, consider:

* **Target audience:** Who are you trying to reach?  A younger audience might respond better to a modern name, while an older audience might prefer something more classic.
* **Brand personality:** Do you want to be seen as luxurious, affordable, quirky, or something else?
* **Memorability:** Is the name easy to remember 

## SimpleSequentialChain

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

# 1. LLMChain
📖 What it is

The most basic chain in LangChain.

It combines:

Prompt Template (defines how user input → model input)

LLM (the engine)

Optional Output Parser (clean the response)

It’s essentially a single-step pipeline.

🛠️ When to Use

You only need one interaction with the LLM.

No dependency on previous steps.

🌍 Real-time Example

Text translation: English → French.

Summarization: Summarize a paragraph.

Classification: Label a review as Positive/Negative.

In [10]:
resume_text = """Sharan is an Tech Lead with 8 years of experience in automation,
Python, FastAPI, SQL, Pytest, Selenium, and building data validation frameworks.
He has experience with GenAI, APIs, cloud services, and leading teams."""

prompt = ChatPromptTemplate.from_template(
    "Summarize the only key skills from this resume:\n{resume}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)
print(llm_chain.invoke({"resume": resume_text})['text'])

Automation, Python, API development (FastAPI), data validation, testing (Pytest, Selenium), team leadership, and GenAI.


## 2. SimpleSequentialChain → Topic → Blog Outline → Blog

Use case: Generate a blog post in 2 steps.

In [14]:
# Step 1: Topic → Outline

prompt1 = ChatPromptTemplate.from_template("Create a blog outline on: {topic}")
chain1 = LLMChain(llm=llm, prompt=prompt1)

# Step 2: Outline → Blog Post
prompt2 = ChatPromptTemplate.from_template("Write a blog from this outline: \n {outline}")
chain2 = LLMChain(llm=llm, prompt=prompt2)

# Sequential pipeline
simple_seq_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

print(simple_seq_chain.run("The future of AI in Development"))



> Entering new SimpleSequentialChain chain...
## Blog Outline: The Future of AI in Development

**I. Introduction (approx. 100 words)**

* Hook: Start with a compelling statistic or anecdote showcasing AI's growing impact on various sectors.  Example:  The rapid advancements in AI are transforming industries, and the development sector is no exception.
* Briefly define "AI in Development" –  focus on its application in tackling global challenges like poverty, climate change, and inequality.
* Thesis statement:  AI holds immense potential to revolutionize development, but its ethical and practical implications require careful consideration for responsible and equitable implementation.
* Briefly outline the blog's structure.


**II. AI's Current Applications in Development (approx. 200 words)**

* **Subheading:**  AI's Present Footprint:  Examples in Action
* **Examples:**
    * **Predictive analytics for disaster relief:** Forecasting natural disasters, optimizing resource allocation.

## 3. SequentialChain → Resume + JD → Tailored Cover Letter

Use case: Match resume against job description and generate cover letter.

### Real-time Example

Input: Candidate’s Resume + Job Description.

Chain 1: Summarize candidate experience. (output → summary)

Chain 2: Compare summary with job description. (inputs → summary + job_desc)

Chain 3: Generate tailored cover letter.

In [16]:
# Job description
job_desc = "Looking for a GEnAI  Engineer skilled in Python, FASTAPI, GenAI, SQL, Selenium, and CI/CD."

# Step 1: Summarize resume
prompt1 = ChatPromptTemplate.from_template(
    "Summarize the candidate's experience: \n {resume}"
)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="summary")

# Step 2: Compare with job description
prompt2 = ChatPromptTemplate.from_template(
    "Compare candidate Summary with job description: \n SUmmary: {summary}\n JD: {jd}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="comparision")

# Step 3: Write cover letter
prompt3 = ChatPromptTemplate.from_template(
    "Write a short cover letter based on summary: \n{summary}\nFor the Job:{jd}"
)
chain3 = LLMChain(llm=llm, prompt=prompt3, output_key="cover_letter")

# Multi-step chain with named inputs/outputs
seq_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=['resume', 'jd'],
    output_variables=["summary", "comparision", "cover_letter"],
    verbose=True
)
result = seq_chain.invoke({"resume": resume_text, "jd": job_desc})
print(result['cover_letter'])



> Entering new SequentialChain chain...

> Finished chain.
Dear [Hiring Manager name],

I am writing to express my keen interest in the GenAI Engineer position advertised on [Platform where you saw the ad].  With eight years of experience as a Tech Lead specializing in software automation, I possess a strong skillset perfectly aligned with your requirements.  My expertise includes Python (including FastAPI and Pytest), SQL, Selenium, data validation frameworks, GenAI, APIs, cloud services, and team leadership.  I am confident in my ability to contribute significantly to your team's success.

My resume provides further detail on my accomplishments and qualifications.  Thank you for your time and consideration.

Sincerely,

Sharan


## 4. RouterChain → Customer Support Bot

Use case: Route queries to the correct department.

#### 🌍 Real-time Example

Customer Support Bot:

Chain A → Billing queries.

Chain B → Technical support.

Chain C → General FAQs.

Router Chain looks at user’s question → decides which chain should handle it.

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch
# Prompts for different departments
billing_prompt = ChatPromptTemplate.from_template(
    "You are a billing assistant. Answer this:\n{input}"
)
tech_prompt = ChatPromptTemplate.from_template(
    "You are a tech support agent. Answer this:\n{input}"
)
faq_prompt = ChatPromptTemplate.from_template(
    "You are an FAQ assistant. Answer this:\n{input}"
)

# Department chains
billing_chain = billing_prompt | llm | StrOutputParser()
tech_chain = tech_prompt | llm | StrOutputParser()
faq_chain = faq_prompt | llm | StrOutputParser()

# Router prompt
router_prompt = ChatPromptTemplate.from_template(
    """Given a customer question, decide which department should handle it.
Options: billing, tech, faq.
Question: {input}
Return ONLY the option name (billing, tech, or faq)."""
)

router_chain = router_prompt | llm | StrOutputParser()

# Routing logic with RunnableBranch
multi_chain = RunnableBranch(
    (lambda x: "billing" in router_chain.invoke(x).lower(), billing_chain),
    (lambda x: "tech" in router_chain.invoke(x).lower(), tech_chain),
    faq_chain,  # default fallback
)

# Try it
print(multi_chain.invoke({"input": "Why was I charged twice?"}))
print(multi_chain.invoke({"input": "My app keeps crashing on startup."}))
print(multi_chain.invoke({"input": "What are your business hours?"}))

There are several reasons why you might have been charged twice. To help me figure out why this happened in your case, could you please provide me with the following information:

* **Your account number or name:** This helps me quickly locate your account.
* **The dates and amounts of the two charges:**  Knowing the exact amounts and when they occurred will pinpoint the issue.
* **The description of the charges:** What do the charges say they are for?  (e.g., "Membership Fee," "Service Charge," etc.)
* **The method of payment used for each charge:** (e.g., credit card, debit card, etc.)

Once I have this information, I can investigate and explain the duplicate charges.  It's possible it was a processing error, a duplicate transaction, or a misunderstanding.  I'll do my best to resolve this for you.
Okay, I understand your app is crashing on startup.  To help me troubleshoot this, I need some more information:

1. **What app is it?** (Name and version number, if possible)
2. **What ope